In [1]:
# some useful mysklearn package import statements and reloads
import importlib

import mysklearn.myutils
importlib.reload(mysklearn.myutils)
import myutils
import mysklearn.mypytable
importlib.reload(mysklearn.mypytable)
from mysklearn.mypytable import MyPyTable 

import mysklearn.myclassifiers
importlib.reload(mysklearn.myclassifiers)
from mysklearn.myclassifiers import MyKNeighborsClassifier, MyDummyClassifier, MyNaiveBayesClassifier, MyDecisionTreeClassifier

import mysklearn.myevaluation
importlib.reload(mysklearn.myevaluation)
import mysklearn.myevaluation as myevaluation

# Car Sales Classifier 

 We are going to perform some eda and use multiple classifiers to detemermine the most likely sale price of a car based on a number of classifiers. Our train set has the following attributes:  

| Attribute     | Type      | Notes                                         |
| :---          |    :----: |:---                                           |
| Sale Price    | int       |Our Class label; will require discretization   |
| Brand         | string    |                                               |
| year          | int       |                                               |
| title status  | string    |                                               |
| mileage       | int       |                                               |
| color         | string    |                                               |
| VIN           | int       |                                               |
| Lot           | int       |                                               |



First we will import out dataset into a table 

In [2]:
auto_dataset = MyPyTable().load_from_file("USA_cars_datasets.csv")
auto_dataset = myutils.clean_data(auto_dataset)
prices = auto_dataset.get_column("price")
myutils.discretize_sales_price(prices)

25.0
84900.0
